In [17]:
import pandas as pd
import scipy.stats as ss

In [19]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')
scores['rougeL'].fmeasure

0.5882352941176471

In [3]:
df_gold = pd.read_csv('../data/mct_combined_v3.csv')
categories = ['Overall'] + list(set(df_gold['question_category']))

In [4]:
df = pd.read_csv('./CounselingQA_metrics_combined.csv')
df = df.sort_values('setting')
df

,setting,Accuracy_Overall,F1_Overall,Precision_Overall,Recall_Overall,"Accuracy_intake, assessment, and diagnosis","F1_intake, assessment, and diagnosis","Precision_intake, assessment, and diagnosis","Recall_intake, assessment, and diagnosis",Accuracy_treatment planning,...,Precision_counseling skills and interventions,Recall_counseling skills and interventions,Accuracy_professional practice and ethics,F1_professional practice and ethics,Precision_professional practice and ethics,Recall_professional practice and ethics,Accuracy_core counseling attributes,F1_core counseling attributes,Precision_core counseling attributes,Recall_core counseling attributes
109,Asclepius-13B_few-shot-cot_None_3,0.2547,0.136640,0.260648,0.219824,0.2792,0.155991,0.275391,0.247780,0.2580,...,0.285845,0.225136,0.2164,0.099963,0.193245,0.197738,0.1828,0.146567,0.283065,0.243939
131,Asclepius-13B_few-shot_None_1,0.2198,0.102242,0.183443,0.182961,0.2611,0.123817,0.194809,0.201491,0.2353,...,0.159550,0.178208,0.1781,0.084735,0.187017,0.158916,0.1339,0.055314,0.034034,0.153555
172,Asclepius-13B_few-shot_None_3,0.3046,0.254857,0.318655,0.245536,0.2938,0.237289,0.299515,0.239864,0.3072,...,0.341625,0.249294,0.3422,0.288360,0.354959,0.273269,0.3527,0.199112,0.212248,0.196926
0,Asclepius-13B_few-shot_sc_1,0.2356,0.105779,0.157158,0.246831,0.2612,0.119659,0.205957,0.245988,0.2398,...,0.128332,0.251264,0.2197,0.095040,0.123063,0.244963,0.1681,0.071558,0.042025,NaN
3,Asclepius-13B_few-shot_sc_3,0.3813,0.372499,0.402487,0.383328,0.3561,0.350120,0.387602,0.369963,0.3600,...,0.378151,0.369728,0.4093,0.402186,0.427424,0.414398,0.4406,0.351308,0.333726,0.399759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,meditron-7b_few-shot_None_5,0.2467,0.231048,0.246470,0.246389,0.2477,0.237789,0.265418,0.252532,0.2698,...,0.220979,0.223772,0.2338,0.215503,0.223144,0.235078,0.3897,0.351525,0.347976,0.364465
25,meditron-7b_few-shot_sc_1,0.2401,0.096460,0.060025,NaN,0.2521,0.100142,0.063025,NaN,0.2312,...,0.061600,NaN,0.2242,0.091012,0.056050,NaN,0.1692,0.071927,0.042300,NaN
179,meditron-7b_few-shot_sc_5,0.2369,0.142294,0.222980,0.247869,0.2183,0.135201,0.185578,0.245725,0.2106,...,0.230948,0.245253,0.2488,0.154178,0.231424,0.248011,0.3944,0.141089,0.098600,NaN
148,meditron-7b_zero-shot_None_0,0.2669,0.215486,0.239386,0.213160,0.2704,0.216378,0.242349,0.217349,0.2639,...,0.213836,0.185641,0.2801,0.223950,0.250756,0.222803,0.1755,0.136647,0.288934,0.144057


In [10]:
# Few-shot helps, only if model is already somewhat capable
for competencies in ['Overall', 'intake, assessment, and diagnosis', 'treatment planning', 'counseling skills and interventions', 'professional practice and ethics', 'core counseling attributes']:
    print(competencies)
    modes = ['few-shot-cot_None_3', 'few-shot-cot_None_1']
    zs_dict = dict([])
    fs_dict = dict([])
    for row in df.iterrows():
        setting = row[1]['setting']
        # if 'Asclepius-13B' in setting or 'counselingQA-gpt4o' in setting or 'Mistral-7B-Instruct-v0.3' in setting or 'Meta-Llama-3-70B-Instruct' in setting or 'Llama-2-70b-hf' in setting:
        if 'MentaLLaMA' in setting:
            continue
        if modes[0] in setting:
            # print(setting)
            zs_dict[setting] = row[1][f'Accuracy_{competencies}']
            # print('@@@')
        elif modes[1] in setting:
            # print(setting)
            fs_dict[setting] = row[1][f'Accuracy_{competencies}']
            # print('###')
    
    zs = []
    fs = []
    for key in fs_dict.keys():
        if key not in zs_dict.keys():
            continue
        if fs_dict[key] > 0.0:
            fs.append(fs_dict[key])
            zs.append(zs_dict[key.replace(modes[1], modes[0])])
    print(zs)
    print(fs)
    print(ss.ttest_rel(zs, fs))

Overall
[]
[]
TtestResult(statistic=nan, pvalue=nan, df=nan)
intake, assessment, and diagnosis
[]
[]
TtestResult(statistic=nan, pvalue=nan, df=nan)
treatment planning
[]
[]
TtestResult(statistic=nan, pvalue=nan, df=nan)
counseling skills and interventions
[]
[]
TtestResult(statistic=nan, pvalue=nan, df=nan)
professional practice and ethics
[]
[]
TtestResult(statistic=nan, pvalue=nan, df=nan)
core counseling attributes
[]
[]
TtestResult(statistic=nan, pvalue=nan, df=nan)


/tmp/ipykernel_2396955/2762640868.py:31: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  print(ss.ttest_rel(zs, fs))
